# Query house matched preference

Assumpt the user has completed several turns of chat with chatbot, and we already get the preference of user.

embedding base chain output by chagpt embeddings and retrieval related houses from lancedb.

first we merge user preference into a list, then we embedding the merged list string and use it to query the lancedb and compare the house's descrption's similarity with user's preference, at last, we compose a prompt by using the user's preference and the retrievaled house list, and ask the chatGPT to rank the house based on users requirements. 

All this step is realized by using langchain.

In [110]:
#
import dotenv
import lancedb
from typing import List
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_community.vectorstores.lancedb import LanceDB
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import NumberedListOutputParser
dotenv.load_dotenv()

# prompt 
user_preference_template = """First, based on users requirements on budget, remove the house that not satisfied.
then ranking the result based on its suitability to users preference. only keep the best 2.
******\n
The users preference are list below:\n{user_require_list} \n
******\n
some related house information is: {context}\n
******\n
{format_instructions}
return result follow the format instructions and each item must be a json obect. 
In each json object, it must includes metadata of that house except vector.
In each json object, it must includes description of that house. 
add new key named 'suitability', the range is [0-1], 1 means all requirements satisfied.
******\n"""
parser = NumberedListOutputParser()

def build_vector_db_query(input: List[str]):
    require_list = ""
    for idx, user_require in enumerate(input["input"]):
        require_list += "{}. {}\n".format(idx+1, user_require)
    return require_list

from langchain_core.runnables import RunnableLambda
merge_pref = RunnableLambda(build_vector_db_query)
# retrieval
db = lancedb.connect('../tmp/lancedb')
table = db.open_table('house_match')

embedding_model = OpenAIEmbeddings()

vectorstore = LanceDB(table, embedding_model, text_key="description")
retrieval = vectorstore.as_retriever(
    search_kwargs = {'k':4}
)

# 1.1 merge input into one string
parse_chain = {"input": RunnablePassthrough()} | merge_pref
# 1.2 retrieval related house
rag = parse_chain | retrieval
# build prompt
prompt = ChatPromptTemplate.from_template(user_preference_template)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

prompt_build = {"context": rag, "user_require_list": parse_chain} | prompt

# generation 
from langchain_openai.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=0)

complete_chain = prompt_build | chat_model | parser

In [111]:
user_answer_example = [
    "I can only afford no more than 150,000 yen/month",
    "I want live near the sea.",
    "I want to live in some place only take 20 minutes to goto Ueno.",
    "I want to work into nearest subway station in 8 minutes.",
    "I hate noise but want to live in downtown area.",
]
output = complete_chain.invoke(user_answer_example)

In [114]:
# result visualize interface
import gradio as gr
import json

def parser_result(s):
    try:
        object = json.loads(s)
        return object
    except Exception as e:
        return {}

markdown_template = """
### House information\n
**{name}** \n
#### Basic Info\n
\n
Location: {location}\t\t\t\tLayout: {layout}\n
Price: {price}\t\t\t\tbuilt year: {year}\n
Description: {desc}\n
\n
"""
with gr.Blocks() as demo:
    for house in output:
        with gr.Column():
            house_meta = parser_result(house)
            gr.Markdown(
                markdown_template.format(
                    name = house_meta["metadata"]["name"],
                    location = house_meta["metadata"]["location"],
                    layout = house_meta["metadata"]["layout"], 
                    price = house_meta["metadata"]["price"], 
                    year = house_meta["metadata"]["year"],
                    desc = house_meta["description"], 
                )
            )
            gr.Label(
                {"suitability": house_meta["suitability"]}
            )

demo.launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
